In [1]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-

In [63]:
import librosa
import librosa.display
import IPython
import numpy as np

from util import save_wav

In [12]:
# 各フィルタをバイナリ形式で開く
filter_names = ['lpf', 'bpf', 'hpf']
filter_dict = {}
for name in filter_names:
    with open('./P_file/'+ name + '.fir', 'rb') as raw:
        # d:C言語でのdouble型，<:リトルエンディアンで読み込む
        filter_dict[name] = np.fromfile(raw, dtype='<d', sep='')

In [18]:
filter_dict['hpf']

array([-2.31975571e-24,  1.15855666e-10, -5.39047081e-23, ...,
       -6.31278196e-10, -5.39047081e-23,  1.15855666e-10])

In [28]:
# バイナリ形式で開く
with open('./P_file/surround_5.1.raw', 'rb') as raw:
    # h:C言語でのshort型，<:リトルエンディアンで読み込む
    raw_data = np.fromfile(raw, dtype='<h', sep='')
    
# L(左前)，C(中央)，R(右前)，LS(左後)，RS(右後)，LFE(スーパーウーファ)
# 上記の全6ch分のデータが格納されているので，それぞれに分ける
left = raw_data[::6]
center = raw_data[1::6]
right = raw_data[2::6]
left_back = raw_data[3::6]
right_back = raw_data[4::6]
woofer = raw_data[5::6]

# 該当式によるステレオ信号を作成
L_o = ((left + 0.5 * center + 0.5 * left_back) / 2.0).astype(np.int16)
R_o = ((right + 0.5 * center + 0.5 * right_back) / 2.0).astype(np.int16)
C = ((L_o + R_o) / 2).astype(np.int16)

In [62]:
for fir, data in filter_dict.items():
    out = []            # 畳み込み演算後の結果を格納
    offset = data.size  # フィルタのサイズ
    
    # 命名
    if fir == 'hpf':
        name = 'high-pass_filter'
    elif fir == 'lpf':
        name = 'low-pass_filter'
    else:
        name = 'band-pass_filter'
        
    # 畳み込み演算
    for i in range(C.size - offset):
        out.append((C[i:i+offset] * data).sum())
    
    # 音源の保存
    out = np.array(out).astype(np.int16)
    save_wav('./P_file/' + name + '.wav', out)